# **카테고리 전처리 항목**
- 특수문자 제거(공백 포함)
- 단순 오탈자 수정 / 용어 통일
- 상위 카테고리 생성

## **0. 라이브러리 및 패키지 import**

In [7]:
import pandas as pd
import re

## **1. 데이터프레임 Load**

In [8]:
# 엑셀 파일 불러오기
df = pd.read_excel("category.xlsx")

## **2. 공백, 특수문자 제거**
- 특수문자 형식이 달라서 모두 삭제 후 쉼표(,)로 변환

In [9]:
# 전처리 함수: 특수문자 제거 + 분리
def clean_and_split_keywords(keyword):
    if pd.isna(keyword):
        return []
    
    # 특수 문자 제거 및 구분자 통일
    cleaned = re.sub(r'[#/]', '', keyword)        # '#'와 '/' 제거
    cleaned = re.sub(r'[·|&]', ',', cleaned)      # '·' 또는 '&' 또는 '|' → ','
    cleaned = cleaned.replace(' ', '')            # 공백 제거
    split_keywords = re.split(r'[,\n]', cleaned)  # 쉼표 또는 줄바꿈으로 분리
    return [kw for kw in split_keywords if kw]

# 분리된 키워드 컬럼 추가
df['분리된_키워드'] = df['키워드'].apply(clean_and_split_keywords)

# 대표 키워드 (리스트의 첫 번째 항목)
df['대표_키워드'] = df['분리된_키워드'].apply(lambda x: x[0] if x else '')

# 출력
df

,키워드,분리된_키워드,대표_키워드
0,#추리/협력,[추리협력],추리협력
1,OX퀴즈,[OX퀴즈],OX퀴즈
2,RPG,[RPG],RPG
3,가족게임,[가족게임],가족게임
4,감각 퍼즐,[감각퍼즐],감각퍼즐
...,...,...,...
354,"협상, 카드","[협상, 카드]",협상
355,협상거래,[협상거래],협상거래
356,협상전략,[협상전략],협상전략
357,화석탐험,[화석탐험],화석탐험


In [10]:
# 저장
df.to_csv("1_clean_keyword.csv", index=False)

## **3. 단순 오탈자 수정 / 용어 통일**
### 단순 오탈자
- 엔진빌 -> 엔진빌딩
- 이야기만들 -> 이야기
- 순발, 리얼타임순발력 -> 순발력

### 용어 통일
- 컬랙션, 컬렉션, 콜랙션 -> 컬렉션
- 배팅 -> 베팅
- 미니어쳐 -> 미니어처
- 셋 -> 세트
- 머더미스터리 -> 머더

In [11]:
def fix_keyword(keyword):
    if pd.isna(keyword):
        return ''

    # 완전 일치 시에만 바꾸는 딕셔너리(단순 오기)
    exact_match_map = {
        '엔진빌': '엔진빌딩',
        '이야기만들': '이야기',
        '순발' : '순발력',
        '리얼타임순발력' : '순발력',
        '복물복' : '복불복'
    }

    # 포함되면 바꾸는 딕셔너리(용어 미통일)
    partial_match_map = {
        '컬랙션' : '컬렉션',
        '콜랙션' : '컬렉션',
        '배팅' : '베팅',
        '미니어쳐' : '미니어처',
        '셋' : '세트',
        '머더미스터리' : '머더'
    }

    # --- 1. 완전 일치 처리 ---
    if keyword in exact_match_map:
        return exact_match_map[keyword]

    # --- 2. 부분 포함 처리 ---
    for wrong, correct in partial_match_map.items():
        if wrong in keyword:
            keyword = keyword.replace(wrong, correct)

    return keyword

# 적용
df['오타_용어_통일'] = df['대표_키워드'].apply(fix_keyword)

# 출력
df

,키워드,분리된_키워드,대표_키워드,오타_용어_통일
0,#추리/협력,[추리협력],추리협력,추리협력
1,OX퀴즈,[OX퀴즈],OX퀴즈,OX퀴즈
2,RPG,[RPG],RPG,RPG
3,가족게임,[가족게임],가족게임,가족게임
4,감각 퍼즐,[감각퍼즐],감각퍼즐,감각퍼즐
...,...,...,...,...
354,"협상, 카드","[협상, 카드]",협상,협상
355,협상거래,[협상거래],협상거래,협상거래
356,협상전략,[협상전략],협상전략,협상전략
357,화석탐험,[화석탐험],화석탐험,화석탐험


In [12]:
# 저장
df.to_csv("2_modify.csv", index=False)

## **3. 상위 카테고리 생성**
- 특정 접두어를 기준으로 범주화
- 접미어를 쓰기엔 범주화 제약이 큼

In [13]:
# 상위 카테고리 추출 함수
def extract_parent_category(keyword):
    parent_keywords = [
        '그림', '카드', '추리', '기억', '전략', '퍼즐', '블록', '협동', '퀴즈',
        '동물', '경제', '단어', '속도', '숫자', '정리', '게임',
        '방', '경매', '공간', '길', '미션', '덱', '실시간', '운빨', '전쟁', '조합', '주사위', '타일', '투자', '파티', '퍼즐',
        '협력', '협상', '공감', '논리', '눈치', '대화', '마을', '마피아', '세트', '순발력', '심리', '영역', '팀전', '패턴', 
        '베팅', '영향력'
    ]
    for parent in parent_keywords:
        if keyword.startswith(parent):
            return parent
    return keyword  # 일치하는 접두사가 없을 경우 그대로 반환

# 적용
df['상위카테고리'] = df['오타_용어_통일'].apply(extract_parent_category)

# 출력
df

,키워드,분리된_키워드,대표_키워드,오타_용어_통일,상위카테고리
0,#추리/협력,[추리협력],추리협력,추리협력,추리
1,OX퀴즈,[OX퀴즈],OX퀴즈,OX퀴즈,OX퀴즈
2,RPG,[RPG],RPG,RPG,RPG
3,가족게임,[가족게임],가족게임,가족게임,가족게임
4,감각 퍼즐,[감각퍼즐],감각퍼즐,감각퍼즐,감각퍼즐
...,...,...,...,...,...
354,"협상, 카드","[협상, 카드]",협상,협상,협상
355,협상거래,[협상거래],협상거래,협상거래,협상
356,협상전략,[협상전략],협상전략,협상전략,협상
357,화석탐험,[화석탐험],화석탐험,화석탐험,화석탐험


In [14]:
# 저장
df.to_csv("3_parent_category.csv", index=False)

## **4. 수기로 전처리해서 상위 카테고리 15개로 범주화**
